Load and pre process **data**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))

for class_id in range(10):
    class_indices = np.where(y_train.flatten() == class_id)[0]
    selected_images = np.random.choice(class_indices, 10, replace=False)

    for i, img_idx in enumerate(selected_images):
        plt.subplot(10, 10, class_id * 10 + i + 1)
        plt.imshow(x_train[img_idx])  # No cmap, as CIFAR-10 is RGB
        plt.axis('off')

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
#Convert the labels to one-hot encoded form.
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print("y_train shape after one-hot encoding:", y_train.shape)
print("y_test shape after one-hot encoding:", y_test.shape)


y_train shape after one-hot encoding: (50000, 10)
y_test shape after one-hot encoding: (10000, 10)


In [ ]:
#Normalize

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print(x_train.shape)
print(x_test.shape)
print("Pixel value range:", x_train.min(), "to", x_train.max())


(50000, 32, 32, 3)
(10000, 32, 32, 3)
Pixel value range: 0.0 to 1.0


In [ ]:
Define model